In [1]:
import pandas as pd
import numpy as np
from glob import glob
from re import match, sub
from missingno import matrix as mmatrix
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from math import ceil
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from statsmodels.formula.api import ols
from os import listdir

from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [2]:
if 'figure_dir.txt' in listdir():
    with open('figure_dir.txt', 'r') as f:
        fig_dir = f.readline().strip().replace('\\', '/')
        tbl_dir = f"{fig_dir.rsplit('/', maxsplit=2)[0]}/Tables/"
else:
    fig_dir = 'Figures/'
    tbl_dir = 'Tables/'

def trim_trailing_zeros(number_string):
    '''
    Take a string with numbers and remove any trailing zeros from the number.
    '''
    return sub(
        r'(?:\.0+|(\.\d+)0+)(%?)$', 
        '\\1\\2', number_string
    )

def neaten_ticks(
        ax, axis, tick_range, ticks, dp=0, fmt='f', twin=False
    ):
    '''
    Take a plot ax and convert its x or y axis into a neatened range frame.
    '''
    # Set sets.
    eval(f"ax.set_{axis}ticks")(sorted([*ticks, *tick_range]))
    is_log = eval(f"ax.get_{axis}scale")() == 'log' 
    # Set labels.
    eval(f"ax.set_{axis}ticklabels")([
        f"$\\mathdefault{{10^{{{ex:.0f}}}}}$" 
        if is_log and int((ex := np.log10(e))) == ex 
        else trim_trailing_zeros(eval(f'''f"{{e:.0{dp}{fmt}}}"'''))
        for e in eval(f"ax.get_{axis}ticks")()
    ])
    # Limit axis outline to data range.
    ax.spines[
        'right' if twin else 'bottom' if axis == 'x' else 'left'
    ].set_bounds(*tick_range)

def show(
    fig, axes=[], tight=False, despine=True, despine_twin=False, 
    delax=True, file=None, legend_loc=0, filetype='pdf'
):
    '''Style and show a matplotlib plot.'''
    if tight: plt.tight_layout() # Neaten subplots.
    # Remove unnecessary outlines from plots with twin axes.
    # Note that this method assumes all axes have twins.
    if despine_twin:
        for ax in axes[::2]:
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
        for ax in axes[1::2]:
            ax.spines['top'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            ax.spines['left'].set_visible(False)
    # Remove unnecessary outlines.
    elif despine:
        for ax in axes:
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
    for ax in axes:
    # Remove unused subplots.
        if delax and not ax.has_data(): fig.delaxes(ax)
        ax.tick_params(
            axis='both', which='minor', 
            bottom=False, left=False, right=False
        )
        # Style the legend if a legend is present.
        if ax.get_legend():
            handles, labels = ax.get_legend_handles_labels()
            labels = [e.title() for e in labels]
            legend_title = ax.get_legend().get_title().get_text().title()
            ax.legend(
                handles=handles, labels=labels, 
                title=legend_title, loc=legend_loc
            )
    # Save file if a filename is provided.
    if file:
        plt.savefig(
            f'{fig_dir}/Plots/{file}.{filetype}', dpi = 300,
            bbox_inches = 'tight', pad_inches = 1/25
        )
    plt.show()
    

In [3]:
CSVs = glob('data/*/*.csv')

for csv in CSVs:
    csv = csv.replace('\\', '/')
    file = match(r'data.*/(.*?)\.csv', csv).group(1)
    print(file)
    exec(f'{file} = pd.read_csv("{csv}")')
    exec(f'{file}["state"] = "{file.split('_')[-1].upper()}"')


forecastdemand_qld
forecastdemand_sa
forecastdemand_vic
temprature_qld
temprature_sa
temprature_vic
totaldemand_qld
totaldemand_sa
totaldemand_vic
forecastdemand_nsw
temperature_nsw
totaldemand_nsw


In [4]:
# totaldemand_nsw
# forecastdemand_nsw
temperature_nsw


,LOCATION,DATETIME,TEMPERATURE,state
0,Bankstown,1/1/2010 0:00,23.1,NSW
1,Bankstown,1/1/2010 0:01,23.1,NSW
2,Bankstown,1/1/2010 0:30,22.9,NSW
3,Bankstown,1/1/2010 0:50,22.7,NSW
4,Bankstown,1/1/2010 1:00,22.6,NSW
...,...,...,...,...
220321,Bankstown,17/3/2021 23:00,19.1,NSW
220322,Bankstown,17/3/2021 23:20,19.0,NSW
220323,Bankstown,17/3/2021 23:30,18.8,NSW
220324,Bankstown,17/3/2021 23:34,18.8,NSW


In [5]:
def get_metadata(data, name):
    return {
        'Dataset': name, 
        'Records': len(data), 
        'Variables': len([e for e in data.columns if e != 'state']),
        'Size in memory (MB)': data.drop(columns=['state']).memory_usage(deep=True).sum() / 1e6
    }

metadata = []


In [6]:
dem = pd.concat([totaldemand_qld, totaldemand_sa, totaldemand_vic])

dem.DATETIME = pd.to_datetime(dem.DATETIME, format="%Y-%m-%d %H:%M:%S")
totaldemand_nsw.DATETIME = pd.to_datetime(totaldemand_nsw.DATETIME, format="%d/%m/%Y %H:%M")
dem = pd.concat([dem, totaldemand_nsw]).reset_index(drop=True)
# dem = dem.drop(columns=['REGIONID'])

dem = dem[dem.state == 'NSW'].reset_index(drop=True)

# print(dem.isnull().sum())

metadata.append(get_metadata(dem, 'totaldemand_nsw'))

dem


,DATETIME,TOTALDEMAND,REGIONID,state
0,2010-01-01 00:00:00,8038.00,NSW1,NSW
1,2010-01-01 00:30:00,7809.31,NSW1,NSW
2,2010-01-01 01:00:00,7483.69,NSW1,NSW
3,2010-01-01 01:30:00,7117.23,NSW1,NSW
4,2010-01-01 02:00:00,6812.03,NSW1,NSW
...,...,...,...,...
196508,2021-03-17 22:00:00,7419.77,NSW1,NSW
196509,2021-03-17 22:30:00,7417.91,NSW1,NSW
196510,2021-03-17 23:00:00,7287.32,NSW1,NSW
196511,2021-03-17 23:30:00,7172.39,NSW1,NSW


In [153]:
fore = pd.concat([
    forecastdemand_qld, forecastdemand_sa, forecastdemand_vic, forecastdemand_nsw
])

fore = fore[fore.state == 'NSW'].reset_index(drop=True)

for col in ["LASTCHANGED", "DATETIME"]:
    fore[col] = pd.to_datetime(fore[col], format="%Y-%m-%d %H:%M:%S")

metadata.append(get_metadata(fore, 'forecastdemand_nsw'))

fore = fore[fore.PERIODID.isin([2, 48])].drop_duplicates()\
    .reset_index(drop=True).drop(columns=['REGIONID'])

print(fore.isnull().sum())

# fore_piv = fore.pivot(columns='PERIODID', index='DATETIME', values='FORECASTDEMAND').reset_index()
# fore_piv.columns = [e if e=='DATETIME' else f"h{e/2:.0f}_ahead" for e in fore_piv.columns]

fore


PREDISPATCHSEQNO    0
PERIODID            0
FORECASTDEMAND      0
LASTCHANGED         0
DATETIME            0
state               0
dtype: int64


,PREDISPATCHSEQNO,PERIODID,FORECASTDEMAND,LASTCHANGED,DATETIME,state
0,2009123041,48,7822.38,2009-12-31 00:01:34,2010-01-01 00:00:00,NSW
1,2009123139,2,7789.50,2009-12-31 23:01:24,2010-01-01 00:00:00,NSW
2,2009123042,48,7715.68,2009-12-31 00:31:25,2010-01-01 00:30:00,NSW
3,2009123140,2,7603.17,2009-12-31 23:31:32,2010-01-01 00:30:00,NSW
4,2009123043,48,7482.56,2009-12-31 01:01:17,2010-01-01 01:00:00,NSW
...,...,...,...,...,...,...
327495,2021031737,2,7316.62,2021-03-17 22:01:34,2021-03-17 23:00:00,NSW
327496,2021031640,48,7011.96,2021-03-16 23:31:34,2021-03-17 23:30:00,NSW
327497,2021031738,2,7187.72,2021-03-17 22:31:36,2021-03-17 23:30:00,NSW
327498,2021031641,48,6932.43,2021-03-17 00:01:34,2021-03-18 00:00:00,NSW


In [8]:
temp = pd.concat([temprature_qld, temperature_nsw, temprature_vic])

temp.DATETIME = pd.to_datetime(temp.DATETIME, format="%d/%m/%Y %H:%M")

temprature_sa.DATETIME = pd.to_datetime(temprature_sa.DATETIME, format="%Y-%m-%d %H:%M:%S")
temp = pd.concat([temp, temprature_sa])\
    .drop(columns = ['Unnamed: 0']).reset_index(drop=True)

temp = temp[temp.state == 'NSW'].reset_index(drop=True)
metadata.append(get_metadata(temp, 'temperature_nsw'))

temp = temp.drop(columns=['LOCATION'])

print(temp.isnull().sum())

temp


DATETIME       0
TEMPERATURE    0
state          0
dtype: int64


,DATETIME,TEMPERATURE,state
0,2010-01-01 00:00:00,23.1,NSW
1,2010-01-01 00:01:00,23.1,NSW
2,2010-01-01 00:30:00,22.9,NSW
3,2010-01-01 00:50:00,22.7,NSW
4,2010-01-01 01:00:00,22.6,NSW
...,...,...,...
220321,2021-03-17 23:00:00,19.1,NSW
220322,2021-03-17 23:20:00,19.0,NSW
220323,2021-03-17 23:30:00,18.8,NSW
220324,2021-03-17 23:34:00,18.8,NSW


In [152]:
df = pd.merge(temp, dem, on=['DATETIME', 'state'], how='outer')\
    .dropna()

# ohe = OneHotEncoder(sparse_output=False)
# df[sorted(df.state.unique())] = ohe.fit_transform(df.state.to_numpy().reshape(-1, 1))
# df = df.drop(columns=['state'])
df['day'] = df.DATETIME.dt.strftime('%w').astype(int)
df['day_label'] = df.DATETIME.dt.strftime('%a')
df['is_weekday'] = df.day.between(1, 5)

h1_ahead = df.copy()[['DATETIME', 'TOTALDEMAND']].rename(columns={'TOTALDEMAND': 'h1_ahead'})
h1_ahead.DATETIME -= timedelta(minutes=30)
h24_ahead = df.copy()[['DATETIME', 'TOTALDEMAND']].rename(columns={'TOTALDEMAND': 'h24_ahead'})
h24_ahead.DATETIME -= timedelta(days=1)

for ahead in [h1_ahead, h24_ahead]:
    df = pd.merge(df, ahead, how='left', on='DATETIME')

df = df.dropna().reset_index(drop=True)

df


,DATETIME,TEMPERATURE,state,TOTALDEMAND,REGIONID,day,day_label,is_weekday,h1_ahead,h24_ahead
0,2010-01-01 00:00:00,23.1,NSW,8038.00,NSW1,5,Fri,True,7809.31,7574.85
1,2010-01-01 00:30:00,22.9,NSW,7809.31,NSW1,5,Fri,True,7483.69,7343.30
2,2010-01-01 01:00:00,22.6,NSW,7483.69,NSW1,5,Fri,True,7117.23,7099.73
3,2010-01-01 01:30:00,22.5,NSW,7117.23,NSW1,5,Fri,True,6812.03,6779.80
4,2010-01-01 02:00:00,22.5,NSW,6812.03,NSW1,5,Fri,True,6544.33,6497.47
...,...,...,...,...,...,...,...,...,...,...
195273,2021-03-16 22:00:00,20.4,NSW,7373.83,NSW1,2,Tue,True,7345.78,7419.77
195274,2021-03-16 22:30:00,20.5,NSW,7345.78,NSW1,2,Tue,True,7218.99,7417.91
195275,2021-03-16 23:00:00,20.3,NSW,7218.99,NSW1,2,Tue,True,7056.88,7287.32
195276,2021-03-16 23:30:00,19.7,NSW,7056.88,NSW1,2,Tue,True,6999.23,7172.39


In [185]:
X_cols = ['TEMPERATURE', 'TOTALDEMAND', 'is_weekday']
y_cols = ['h1_ahead', 'h24_ahead']
X_y_df = df.dropna()[X_cols + y_cols].astype(float).dropna()
X_df = X_y_df.drop(columns=y_cols)
y_df = X_y_df[y_cols]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_df)

X_y = [X_train, X_test, y_train, y_test] = train_test_split(
    X_scaled, y_df, test_size=0.2, shuffle=False
)
X_y = [tf.convert_to_tensor(d) for d in X_y]
[X_train, X_test, y_train, y_test] = X_y

train_val_I = TimeSeriesSplit(n_splits=10).split(X_train)

for train, val in train_val_I:
    print(train.shape, val.shape)

X_train


(14202,) (14202,)
(28404,) (14202,)
(42606,) (14202,)
(56808,) (14202,)
(71010,) (14202,)
(85212,) (14202,)
(99414,) (14202,)
(113616,) (14202,)
(127818,) (14202,)
(142020,) (14202,)


<tf.Tensor: shape=(156222, 3), dtype=float64, numpy=
array([[ 0.94650813, -0.05568891,  0.63279108],
       [ 0.91250576, -0.23186237,  0.63279108],
       [ 0.86150219, -0.48270677,  0.63279108],
       ...,
       [-0.39658566, -1.3307491 ,  0.63279108],
       [-0.17557022, -1.17549089,  0.63279108],
       [ 0.07944758, -1.04939077,  0.63279108]])>

In [85]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier


In [186]:
main_input = Input(shape=tuple(X_train[0].shape), name="input")
previous_layer = main_input
for i in range(5):
    # previous_layer = BatchNormalization()(previous_layer)
    previous_layer = Dense(32, name=f"linear_{i+1}", activation='linear')(previous_layer)
    previous_layer = Dense(32, name=f"relu_{i+1}", activation='relu')(previous_layer)
main_output = Dense(2, name=f"output")(previous_layer)

model_mlp = Model(inputs=main_input, outputs=main_output, name="MLP")
model_mlp.compile(optimizer=Adam(
    learning_rate=.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    name='Adam',
), loss=MeanSquaredError(), metrics=[MAPE])
model_mlp.fit(X_train, y_train, epochs=1000, batch_size=2**5, verbose=1, callbacks=[EarlyStopping(monitor='loss', patience=3)])



Epoch 1/1000
4882/4882 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 5232846.5000 - mean_absolute_percentage_error: 12.2667
Epoch 2/1000
4882/4882 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 233583.2812 - mean_absolute_percentage_error: 4.0095
Epoch 3/1000
4882/4882 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 228432.4062 - mean_absolute_percentage_error: 3.9760
Epoch 4/1000
4882/4882 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 219762.9531 - mean_absolute_percentage_error: 3.9103
Epoch 5/1000
4882/4882 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 218091.2969 - mean_absolute_percentage_error: 3.9006
Epoch 6/1000
4882/4882 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 216861.4219 - mean_absolute_percentage_error: 3.8711
Epoch 7/1000
4882/4882 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 213503.4062 - mean_absolute_percentage_error: 3.8399
Epoch 8/1000
4882/4882 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 213368.4062 - mean_absolute_percentage_error: 3.8239
Epoch 9/1000
4882/4882 ━━━━━━━━━━━━━━━━━━━━ 18s 4ms/st

In [219]:
y_pred = model_mlp.predict(X_test)
for i in range(2):
    print(('H24' if i else 'H1'.rjust(3)) + ':', r2_score(y_test[:,i], y_pred[:,i]))


 H1: 0.9689171128440581
H24: 0.7387853371008852


In [180]:
from statsmodels.formula.api import ols

model = ols("h1_ahead ~ TEMPERATURE + I(TEMPERATURE**2) + I(TEMPERATURE**3) + I(TEMPERATURE**4) + is_weekday + TOTALDEMAND + I(TOTALDEMAND**2) + I(TOTALDEMAND**3) + I(TOTALDEMAND**4)", X_y_df).fit()

model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               h1_ahead   R-squared:                       0.972
Model:                            OLS   Adj. R-squared:                  0.972
Method:                 Least Squares   F-statistic:                 1.356e+06
Date:                Fri, 22 Mar 2024   Prob (F-statistic):               0.00
Time:                        17:32:36   Log-Likelihood:            -1.3279e+06
No. Observations:              195278   AIC:                         2.656e+06
Df Residuals:                  195272   BIC:                         2.656e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.0005   3.47e-06    143.334      0.000       0.000       0.001
TEMPERATURE            -0.0827      0.004    -23.520      0.000      -0.090      -0.076
I(TEMPERATURE ** 2)    -0.9129      0.038    -24.225      0.000      -0.987      -0.839
I(TEMPERATURE ** 3)     0.0453      0.002     18.705      0.000       0.041       0.050
I(TEMPERATURE ** 4)    -0.0004   4.14e-05    -10.755      0.000      -0.001      -0.000
is_weekday             -0.0011      5e-05    -22.043      0.000      -0.001      -0.001
TOTALDEMAND             1.1057      0.007    147.933      0.000       1.091       1.120
I(TOTALDEMAND ** 2) -2.724e-05   2.54e-06    -10.736      0.000   -3.22e-05   -2.23e-05
I(TOTALDEMAND ** 3)  2.637e-09   2.83e-10      9.325      0.000    2.08e-09    3.19e-09
I(TOTALDEMAND ** 4) -9.495e-14   1.03e-14     -9.187      0.000   -1.15e-13   -7.47e-14
==============================================================================
Omnibus:                    31726.294   Durbin-Watson:                   0.338
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            56198.583
Skew:                           1.052   Prob(JB):                         0.00
Kurtosis:                       4.576   Cond. No.                     3.33e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.33e+18. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
model.evaluate(X_val,  y_val, verbose=2)


4526/4526 - 6s - 1ms/step - accuracy: 0.0000e+00 - loss: 714211.7500 - mse: 714328.1250


[714211.75, 0.0, 714328.125]

In [26]:
y_pred = model.predict(X_val)

r2_score(y_val, y_pred)


4526/4526 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step


0.9016066039647717

In [29]:
from tensorflow import keras # for building Neural Networks
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Bidirectional, GRU, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network


In [55]:
start = datetime.now()

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

mlp = MLPRegressor(
    hidden_layer_sizes=(10, 10, 10, 10, 10), solver='adam', 
    max_iter=1000, random_state=1
)
mlp.fit(X=X_train_scaled, y=y_train)
y_pred = mlp.predict(X_val_scaled)

print('Train time:', datetime.now() - start)
print('R-squared:', r2_score(y_val, y_pred))

mlp


Train time: 0:00:19.339882
R-squared: 0.9947232481914644


MLPRegressor(hidden_layer_sizes=(10, 10, 10, 10, 10), max_iter=1000,
             random_state=1)

**MLPRegressor(hidden_layer_sizes=(10, 10, 10), max_iter=1000, random_state=1)**  
Train time: 0:02:50.626288  
R-squared: 0.903996701869163  

**MLPRegressor(hidden_layer_sizes=(30, 30, 30), max_iter=1000, random_state=1)**  
Train time: 0:07:13.764777  
R-squared: 0.9029300360828791  

  
**MLPRegressor(hidden_layer_sizes=(10, 10), max_iter=1000, random_state=1)**  
Train time: 0:04:49.145756  
R-squared: 0.904020785674071  

**MLPRegressor(hidden_layer_sizes=(100), max_iter=1000, random_state=1)**  
Train time: 0:30:34.488262  
R-squared: 0.9042026519524867  
  
**MLPRegressor(hidden_layer_sizes=(1,), max_iter=1000, random_state=1)**  
Train time: 0:15:16.664206  
R-squared: -1.2258165361865667  

**MLPRegressor(hidden_layer_sizes=(5, 5, 5), max_iter=1000, random_state=1)**  
Train time: 0:07:20.061461  
R-squared: 0.9029303895811518  
  
**MLPRegressor(hidden_layer_sizes=(10, 10, 10, 10, 10), max_iter=1000, random_state=1)**  
Train time: 0:00:54.204514  
R-squared: 0.903889951968013  

**MLPRegressor(hidden_layer_sizes=(10, 10, 10, 10, 10, 10, 10, 10, 10, 10), max_iter=1000, random_state=1)**  
Train time: 0:02:17.239752  
R-squared: 0.8997796694909751  

**MLPRegressor(hidden_layer_sizes=(10, 10, 10, 10, 10), max_iter=1000, random_state=1, solver='sgd')**  
Train time: 0:10:37.488748  
R-squared: -2.325212494791451e-05  




In [121]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from urllib.parse import quote_plus


In [132]:
uri = "mongodb+srv://<user>:<password>@project-data.cfluj8d.mongodb.net/?retryWrites=true&w=majority&appName=project-data"\
    .replace('<user>', quote_plus(user)).replace('<password>', quote_plus(psw))

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
client.admin.command('ping')
print("Pinged your deployment. You successfully connected to MongoDB!")



Pinged your deployment. You successfully connected to MongoDB!


In [207]:
print(client.list_database_names())


['data', 'admin']
['forecast_demand', 'total_demand', 'temperature']


In [208]:
# client['data']['temperature'].insert_many(temp.to_dict(orient='records'))
# client['data']['total_demand'].insert_many(dem.to_dict(orient='records'))
# client['data']['forecast_demand'].insert_many(fore.to_dict(orient='records'))


In [210]:
for col in client['data'].list_collection_names():
    print(f"{col}: {client['data'][col].count_documents({}):,} records")
    

forecast_demand: 23,192,795 records
total_demand: 786,051 records
temperature: 778,177 records
